# Monte Carlo World Cup Simulation

### Elo Data
###### Source date: 05/22/2018 
https://www.eloratings.net/

In [3]:
participant_elos = {'Australia':1714, 
                    'Iran':1796,
                    'Japan':1692, 
                    'Saudi Arabia': 1600,
                    'South Korea':1745, 
                    'Egypt':1646,
                    'Morocco':1711,
                    'Nigeria':1699,
                    'Senegal':1747,
                    'Tunisia':1649,
                    'Costa Rica':1745,
                    'Mexico':1859,
                    'Panama':1669,
                    'Argentina': 1985,
                    'Brazil':2131,
                    'Colombia':1935,
                    'Peru':1906,
                    'Uruguay':1891,
                    'Belgium':1931,
                    'Croatia':1853,
                    'Denmark':1843,
                    'England':1941,
                    'France':1984,
                    'Germany':2092,
                    'Iceland':1787,
                    'Poland':1831,
                    'Portugal':1975,
                    'Russia':1685,       # Host Country
                    'Serbia':1770,
                    'Spain':2049,
                    'Sweden':1796,
                    'Switzerland':1879              
                   }

In [4]:
groups = {'Group_A': ['Russia', 'Saudi Arabia', 'Egypt', 'Uruguay'],
          'Group_B': ['Portugal', 'Spain', 'Morocco', 'Iran'],
          'Group_C': ['France', 'Australia', 'Peru', 'Denmark'],
          'Group_D': ['Argentina', 'Iceland', 'Croatia', 'Nigeria'],
          'Group_E': ['Brazil', 'Switzerland', 'Costa Rica', 'Serbia'],
          'Group_F': ['Germany', 'Mexico', 'Sweden', 'South Korea'],
          'Group_G': ['Belgium', 'Panama', 'Tunisia', 'England'],
          'Group_H': ['Poland', 'Senegal', 'Colombia', 'Japan']       
        }

#### Win probability formula
$$W_e = \frac{1}{10^{\frac{-d_r}{400}} + 1}$$


In [60]:
from itertools import combinations
import random

def binomial_trial(success_rate):
    if success_rate > random.random():
        return True
    return False

def win_probability(team, opponent, elo=participant_elos):
    team_elo = elo[team]
    opponent_elo = elo[opponent]
    return (1 / (10 ** ( - ( team_elo-opponent_elo) / 400) + 1 ) )

def stochastic_game_winner(team, opponent, elo=participant_elos):
    team_win_rate = win_probability(team, opponent, elo)
    if binomial_trial(team_win_rate):
        return team
    
    return opponent

def single_group_run(group_name):
    group_members = groups[group_name]
    final_scores = {k:0 for k in group_members}
    for game in combinations(group_members, 2):
        win_prob = win_probability(game[0], game[1], participant_elos)
        if binomial_trial(win_prob):
            final_scores[game[0]] += 1
        else:
            final_scores[game[1]] += 1
            
    rank = sorted(final_scores, key=lambda k: final_scores[k], reverse=True)
    return rank[0], rank[1]

def calculate_group_outcome_probabilities(group_name, number_of_runs):
    group_members = groups[group_name]
    first_place_count = {k:0 for k in group_members}
    second_place_count = {k:0 for k in group_members}
    for run in range(number_of_runs):
        first, second = single_group_run(group_name)
        first_place_count[first] += 1
        second_place_count[second] += 1
        
    first_place_normalized = {key:float(value)/sum(first_place_count.values()) for (key,value) in first_place_count.items()}
    second_place_normalized = {key:float(value)/sum(second_place_count.values()) for (key,value) in second_place_count.items()}
        
    return first_place_normalized, second_place_normalized

###### 8ths: Winner C vs Second D | Winner A vs Second B | Winner B vs Second A | Winner D vs Second C | Winner E vs Second F | Winner G vs Second E | Winner F vs Second E | Winner H vs Second G

###### quarter-finals: 

![image.png](img/match_paths.png)


In [64]:
def world_cup_single_run():
    a_first, a_second = single_group_run('Group_A')
    b_first, b_second = single_group_run('Group_B')
    c_first, c_second = single_group_run('Group_C')
    d_first, d_second = single_group_run('Group_D')
    e_first, e_second = single_group_run('Group_E')
    f_first, f_second = single_group_run('Group_F')
    g_first, g_second = single_group_run('Group_G')
    h_first, h_second = single_group_run('Group_H')
    
    sixteen_1 = stochastic_game_winner(a_first, b_second)
    sixteen_2 = stochastic_game_winner(c_first, d_second)
    sixteen_3 = stochastic_game_winner(e_first, f_second)
    sixteen_4 = stochastic_game_winner(g_first, h_second)
    sixteen_5 = stochastic_game_winner(b_first, a_second)
    sixteen_6 = stochastic_game_winner(d_first, c_second)
    sixteen_7 = stochastic_game_winner(f_first, e_second)
    sixteen_8 = stochastic_game_winner(h_first, g_second)
    
    quarter_final_1 = stochastic_game_winner(sixteen_1, sixteen_2)
    quarter_final_2 = stochastic_game_winner(sixteen_3, sixteen_4)
    quarter_final_3 = stochastic_game_winner(sixteen_5, sixteen_6)
    quarter_final_4 = stochastic_game_winner(sixteen_7, sixteen_8)
    
    semi_final_1 = stochastic_game_winner(quarter_final_1, quarter_final_2)
    semi_final_2 = stochastic_game_winner(quarter_final_3, quarter_final_4)
    
    winner = stochastic_game_winner(semi_final_1, semi_final_2)
    
    return winner

def calculate_world_cup_outcome_probabilities(number_of_runs):
    countries = participant_elos.keys()
    wins_total = {k:0 for k in countries}
    
    for run in range(number_of_runs):
        winning_country = world_cup_single_run()
        wins_total[winning_country] += 1
        
    return wins_total
    
calculate_world_cup_outcome_probabilities(10000)
    

{'Argentina': 613,
 'Australia': 6,
 'Belgium': 320,
 'Brazil': 2971,
 'Colombia': 292,
 'Costa Rica': 7,
 'Croatia': 63,
 'Denmark': 31,
 'Egypt': 0,
 'England': 327,
 'France': 626,
 'Germany': 1987,
 'Iceland': 33,
 'Iran': 14,
 'Japan': 1,
 'Mexico': 70,
 'Morocco': 1,
 'Nigeria': 2,
 'Panama': 4,
 'Peru': 169,
 'Poland': 72,
 'Portugal': 642,
 'Russia': 5,
 'Saudi Arabia': 1,
 'Senegal': 15,
 'Serbia': 8,
 'South Korea': 3,
 'Spain': 1442,
 'Sweden': 10,
 'Switzerland': 122,
 'Tunisia': 0,
 'Uruguay': 143}